In [16]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil
import requests
from tqdm import tqdm
from glob import glob

from pycocotools.coco import COCO

# 원본 train_data 경로
input_path  = './data/train'
# 출력물 저장 경로
output_path = './data/labels_YOLO'

# instantiate COCO specifying the annotations json path
coco = COCO('./data/labels/train.json')

# Specify a list of category names of interest
coco_class = ['농어', '베스', '숭어', '강준치', '블루길', '잉어', '붕어', '누치']


f = open('./data/labels/train.json')
data = json.load(f)
f.close()

"""
농어, 베스, 숭어, 강준치, 블루길, 잉어, 붕어, 누치
"""

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


'\n농어, 베스, 숭어, 강준치, 블루길, 잉어, 붕어, 누치\n'

In [19]:
# 전체 데이터에서 물고기가 존재하는 파일만 따로 뺀게 YOLO/train.
# YOLO/train을 먼저 만들어야함.

data_dir = "./data/"

train_dir = data_dir + "train"
test_dir  = data_dir + "test"

train_re_dir = data_dir + "YOLO/train/"
test_re_dir = data_dir + "YOLO/test/"


train_list = os.listdir(train_dir)
test_list  = os.listdir(test_dir)


train_re_list = glob(f"{train_re_dir}*.png")
test_re_list = glob(f"{test_re_dir}*.png")


train_list = sorted(train_list, key= lambda x: int(x[6:-4]))
test_list  = sorted(test_list, key= lambda x: int(x[5:-4]))


print("* Original train len")
print(f"train : {len(train_list)}, test : {len(test_list)}")
print("-----------------------------")
print("* Re-dataset train len")
print(f"train : {len(train_re_list)}, test : {len(test_re_list)}")

/home/yebu/ai_competitions/aifactory/fish_copy
* Original train len
train : 104875, test : 44946
-----------------------------
* Re-dataset train len
train : 4683, test : 1169


In [6]:
ann_json    = data_dir + 'labels/train.json'
answer_json = data_dir + 'labels/answer_sample.json'

coco = COCO(ann_json)
coco_answer = COCO(answer_json)

whole_image_ids = coco.getImgIds()
image_ids = []
no_anno_ids = []

# to remove not annotated image idx
for idx in whole_image_ids:
    annotations_ids = coco.getAnnIds(imgIds=idx, iscrowd=False)
    if len(annotations_ids) == 0:
        no_anno_ids.append(idx)
    else:
        image_ids.append(idx)


train_files = []
for id in image_ids:
    train_file = coco.loadImgs(id)[0]['file_name']
    train_files.append(train_file)

print("train_files: ", len(train_files))

loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
train_files:  5561


In [10]:
import shutil

output_path = './data/train_fish'

def load_images_from_folder(folder, train_files, output_path):
    count = 0
    for filename in tqdm(train_files):
        source = os.path.join(folder, filename)
        destination = f"{output_path}/{filename[:-4]}.jpg"
        
        try:
            shutil.copy(source, destination)
            # print("File copied successfully.")

        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        count += 1

load_images_from_folder(folder='./data/train', train_files=train_files, output_path=output_path)

100%|██████████| 5561/5561 [00:05<00:00, 958.87it/s] 


In [11]:
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

In [13]:
from glob import glob

target_png = r"./data/yolo_train/*.png"
target_txt = r"./data/yolo_train/*.txt"

png_list = glob(target_png)
txt_list = glob(target_txt)

png_list[:5], txt_list[:5]

([], [])

In [5]:
des_dir = "./data/yolo_val"

for cnt, i in enumerate(zip(png_list, txt_list)):
    if cnt == 1000: break

    img, txt = i[0], i[1]

    shutil.move(img, des_dir)
    shutil.move(txt, des_dir)

In [ ]:
cnt = 0
for txt_file in glob_list:

    with open(txt_file, 'r') as f:
        length_check = []
        for line in f:
            length_check.append(line)
        
        if len(length_check) != 1:
            cnt += 1
            print('check file : ', txt_file, len(length_check))
            
print(cnt)

In [16]:
with open("/USER/mnc/fish/data/yolo_train/train_68990.txt", 'r') as f:
    for i in f:
        print(i)

2 0.2722734 0.6179895 0.2723906 0.1932291

2 0.3991328 0.4901041 0.2711718 0.1585833

2 0.3845937 0.5598125 0.2537187 0.1584166

